In [1]:
import numpy as np
from scipy.optimize import minimize



C:\Users\afmb\Anaconda3\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\afmb\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
C:\Users\afmb\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


# The following code is just min max latency

In [31]:
def objective_function(x, a, b, c, d, p, n_bits):
    # Calculate the value of each function f_i(x)
    fi_values = [n_bits / np.log2(1 + ai * xi / (bi * xi + np.sum(cij * pj for j, pj in enumerate(p)) + di)) for ai, bi, cij, di, xi in zip(a, b, c, d, x)]
    # Return the maximum of the fi_values
    return -np.max(fi_values)  # We negate because scipy's minimize finds the minimum

def constraint_function(x):
    # Box constraints: 0 <= x_i <= 1
    return np.clip(x, 0, 1) - x

# Example values for a, b, c, d
a = np.array([4, 4, 3])  # Example coefficients for a_i
b = np.array([1, 2, 1])  # Example coefficients for b_i
c = np.array([[0.1, 0.08, 0.03], [0.02, 0.3, 0.04], [0.13, 0.4, 0.85]])  # Example coefficients for c_ij
d = np.array([0.2, 0.07, 0.1])  # Example coefficients for d_i

n_bits = np.array([1, 1, 1])  # bits 


# Initial guess for x
x0 = np.random.rand(len(a))

# Bounds for x: 0 <= x_i <= 1
bounds = [(0, 1)] * len(a)

# Define the optimization problem
problem = {
    'fun': objective_function,
    'x0': x0,
    'args': (a, b, c, d, np.zeros_like(a), n_bits),  # Assuming p is initially zeros
    'constraints': {'type': 'ineq', 'fun': constraint_function},
    'bounds': bounds
}

# Solve the optimization problem
result = minimize(**problem, method='trust-constr')

# Extract the optimal solution
optimal_solution = result.x

print("Optimal solution:", optimal_solution)

K = 3
ell = np.zeros([K,1])
E = np.zeros([K,1])
x = optimal_solution
for i in range(K):
    b_prim = 0
    for j in range(K):
        b_prim += c[i, j] * x[j]
    
    b_prim -= c[i,i]*x[i]    
        
    ell[i] = n_bits[i] / np.log2(1 + a[i] * x[i] / (b[i] * x[i] + b_prim + d[i]))
    
    E[i] = x_u*x[i]*ell[i]
    
    
print('latency:', ell) 
print('----------------')
print('energy:', E)

C:\Users\afmb\AppData\Local\Temp\ipykernel_9324\693247915.py:3: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  fi_values = [n_bits / np.log2(1 + ai * xi / (bi * xi + np.sum(cij * pj for j, pj in enumerate(p)) + di)) for ai, bi, cij, di, xi in zip(a, b, c, d, x)]
C:\Users\afmb\AppData\Local\Temp\ipykernel_9324\693247915.py:3: RuntimeWarning: invalid value encountered in log2
  fi_values = [n_bits / np.log2(1 + ai * xi / (bi * xi + np.sum(cij * pj for j, pj in enumerate(p)) + di)) for ai, bi, cij, di, xi in zip(a, b, c, d, x)]


ValueError: array must not contain infs or NaNs

In [148]:
print("Optimal solution:", optimal_solution)


Optimal solution: [4.56540365e-01 4.92457035e-16 5.31645899e-01]


# The following code is with min (sum energy + max latency)

In [59]:
import numpy as np
from scipy.optimize import minimize

def objective_function(x, a, b, c, d, p, n_bits, x_u):
    # Compute the sum of functions for each x_k
    sum_term = sum(n_bits[k]*x[k]*x_u / np.log2(1 + a[k] * x[k] / (b[k] * x[k] + np.sum(c[k, j] * x[j] for j in range(len(x))) - c[k, k] * x[k] + d[k])) for k in range(len(x)))
    # Compute the maximum of the second term
    max_term = max(n_bits[i] / np.log2(1 + a[i] * x[i] / (b[i] * x[i] + np.sum(c[i, j] * x[j] for j in range(len(x)))  - c[i, i] * x[i] + d[i])) for i in range(len(x)))
    return max_term + sum_term 

def constraint_function(x):
    # Box constraints: 0 <= x_i <= 1
    return np.clip(x, 0, 1) - x

# Example values for a, b, c, d, p, n_bits, x_u
x_u = 0.1
a = np.array([2, 8, 5])  
b = np.array([3, 2, 1])  
c = np.array([[0.12, 0.2, 0.13], [0.2, 0.03, 0.04], [0.3, 0.4, 0.5]])  
d = np.array([0.1, 0.2, 0.3])  
p = np.zeros_like(a)  # Example values for p
n_bits = np.array([5, 9, 9])  # Example values for B

# Initial guess for x
x0 = np.random.rand(len(a))

# Bounds for x: 0 <= x_i <= 1
bounds = [(0, 1)] * len(a)

# Define the optimization problem
problem = {
    'fun': objective_function,
    'x0': x0,
    'args': (a, b, c, d, p, n_bits, x_u),
    'constraints': {'type': 'ineq', 'fun': constraint_function},
    'bounds': bounds
}

# Solve the optimization problem: # Good methods: 1) trust-constr, 2) SLSQP, 3) Powell
result = minimize(**problem, method='trust-constr', options={'maxiter': 10000}) 

# Extract the optimal solution
optimal_solution = result.x

print("Optimal solution:", optimal_solution)
K = 3
ell = np.zeros([K,1])
E = np.zeros([K,1])
x = optimal_solution
for i in range(K):
    b_prim = 0
    for j in range(K):
        b_prim += c[i, j] * x[j]
    
    b_prim -= c[i,i]*x[i]    
        
    ell[i] = n_bits[i] / np.log2(1 + a[i] * x[i] / (b[i] * x[i] + b_prim + d[i]))
    
    E[i] = x_u*x[i]*ell[i]
    
    
print('latency:', ell) 
print('max latency:', np.max(ell))
print('----------------')
print('energy:', E)

print('sum energy:', np.sum(E))

C:\Users\afmb\AppData\Local\Temp\ipykernel_9324\3950755041.py:6: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum_term = sum(n_bits[k]*x[k]*x_u / np.log2(1 + a[k] * x[k] / (b[k] * x[k] + np.sum(c[k, j] * x[j] for j in range(len(x))) - c[k, k] * x[k] + d[k])) for k in range(len(x)))
C:\Users\afmb\AppData\Local\Temp\ipykernel_9324\3950755041.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  max_term = max(n_bits[i] / np.log2(1 + a[i] * x[i] / (b[i] * x[i] + np.sum(c[i, j] * x[j] for j in range(len(x)))  - c[i, i] * x[i] + d[i])) for i in range(len(x)))


Optimal solution: [0.54483888 0.08211653 0.18668836]
latency: [[7.24124399]
 [7.24124397]
 [7.24124396]]
max latency: 7.241243986944373
----------------
energy: [[0.39453112]
 [0.05946258]
 [0.13518559]]
sum energy: 0.5891793006639037


In [60]:
#print("Optimal solution:", optimal_solution)

#print('latency:', ell)
#print('max latency:', np.max(ell))
#print('----------------')
#print('energy:', E)

#print('sum energy:', np.sum(E))

# In the following, I removed log2 and repeat the optimization

In [61]:
import numpy as np
from scipy.optimize import minimize

def objective_function(x, a, b, c, d, p, n_bits, x_u):
    # Compute the sum of functions for each x_k
    sum_term = sum(n_bits[k]*x[k]*x_u / np.log2(1 + a[k] * x[k] / (b[k] * x[k] + np.sum(c[k, j] * x[j] for j in range(len(x))) - c[k, k] * x[k] + d[k])) for k in range(len(x)))
    # Compute the maximum of the second term
    max_term = max(n_bits[i] /( a[i] * x[i] / (b[i] * x[i] + np.sum(c[i, j] * x[j] for j in range(len(x))) - c[i, i] * x[i] + d[i])) for i in range(len(x)))
    return max_term + sum_term 

def constraint_function(x):
    # Box constraints: 0 <= x_i <= 1
    return np.clip(x, 0, 1) - x

# Example values for a, b, c, d, p, n_bits, x_u
x_u = 0.1
a = np.array([2, 8, 5])  
b = np.array([3, 2, 1])  
c = np.array([[0.12, 0.2, 0.13], [0.2, 0.03, 0.04], [0.3, 0.4, 0.5]])  
d = np.array([0.1, 0.2, 0.3])  
p = np.zeros_like(a)  # Example values for p
n_bits = np.array([5, 9, 9])  # Example values for B

# Initial guess for x
x0 = np.random.rand(len(a))

# Bounds for x: 0 <= x_i <= 1
bounds = [(0, 1)] * len(a)

# Define the optimization problem
problem = {
    'fun': objective_function,
    'x0': x0,
    'args': (a, b, c, d, p, n_bits, x_u),
    'constraints': {'type': 'ineq', 'fun': constraint_function},
    'bounds': bounds
}

# Solve the optimization problem: # Good methods: 1) trust-constr, 2) SLSQP, 3) Powell
result = minimize(**problem, method='trust-constr', options={'maxiter': 10000}) 

# Extract the optimal solution
optimal_solution = result.x

print("Optimal solution:", optimal_solution)
K = 3
ell = np.zeros([K,1])
E = np.zeros([K,1])
x = optimal_solution
for i in range(K):
    b_prim = 0
    for j in range(K):
        b_prim += c[i, j] * x[j]
    
    b_prim -= c[i,i]*x[i]    
        
    ell[i] = n_bits[i] / np.log2(1 + a[i] * x[i] / (b[i] * x[i] + b_prim + d[i]))
    
    E[i] = x_u*x[i]*ell[i]
    
    
print('latency:', ell) 
print('max latency:', np.max(ell))
print('----------------')
print('energy:', E)

print('sum energy:', np.sum(E))

C:\Users\afmb\AppData\Local\Temp\ipykernel_9324\4150679023.py:6: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum_term = sum(n_bits[k]*x[k]*x_u / np.log2(1 + a[k] * x[k] / (b[k] * x[k] + np.sum(c[k, j] * x[j] for j in range(len(x))) - c[k, k] * x[k] + d[k])) for k in range(len(x)))
C:\Users\afmb\AppData\Local\Temp\ipykernel_9324\4150679023.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  max_term = max(n_bits[i] /( a[i] * x[i] / (b[i] * x[i] + np.sum(c[i, j] * x[j] for j in range(len(x))) - c[i, i] * x[i] + d[i])) for i in range(len(x)))


Optimal solution: [0.6931811  0.06767987 0.15576969]
latency: [[7.1260077 ]
 [8.26343399]
 [8.26343397]]
max latency: 8.263433987635313
----------------
energy: [[0.49396138]
 [0.05592682]
 [0.12871926]]
sum energy: 0.6786074589310795


# The following is the wcnc approach

In [73]:
import numpy as np
from scipy.optimize import minimize

def objective_function(x, a, b, c, d, p, n_bits, x_u):
    # Compute the sum of functions for each x_k
    sum_term = 1*sum(n_bits[k]*x[k]*x_u / np.log2(1 + a[k] * x[k] / (b[k] * x[k] + np.sum(c[k, j] * x[j] for j in range(len(x))) - c[k, k] * x[k] + d[k])) for k in range(len(x)))
    # Compute the maximum of the second term
    max_term = 0.5*np.sum(n_bits[i] /np.log2(1+ a[i] * x[i] / (b[i] * x[i] + np.sum(c[i, j] * x[j] for j in range(len(x))) - c[i, i] * x[i] + d[i])) for i in range(len(x)))
    return max_term + sum_term 

def constraint_function(x):
    # Box constraints: 0 <= x_i <= 1
    return np.clip(x, 0, 1) - x

# Example values for a, b, c, d, p, n_bits, x_u
x_u = 0.1
a = np.array([2, 8, 5])  
b = np.array([3, 2, 1])  
c = np.array([[0.12, 0.2, 0.13], [0.2, 0.03, 0.04], [0.3, 0.4, 0.5]])  
d = np.array([0.1, 0.2, 0.3])  
p = np.zeros_like(a)  # Example values for p
n_bits = np.array([5, 9, 9])  # Example values for B

# Initial guess for x
x0 = np.random.rand(len(a))

# Bounds for x: 0 <= x_i <= 1
bounds = [(0, 1)] * len(a)

# Define the optimization problem
problem = {
    'fun': objective_function,
    'x0': x0,
    'args': (a, b, c, d, p, n_bits, x_u),
    'constraints': {'type': 'ineq', 'fun': constraint_function},
    'bounds': bounds
}

# Solve the optimization problem: # Good methods: 1) trust-constr, 2) SLSQP, 3) Powell
result = minimize(**problem, method='trust-constr', options={'maxiter': 10000}) 

# Extract the optimal solution
optimal_solution = result.x

print("Optimal solution:", optimal_solution)
K = 3
ell = np.zeros([K,1])
E = np.zeros([K,1])
x = optimal_solution
for i in range(K):
    b_prim = 0
    for j in range(K):
        b_prim += c[i, j] * x[j]
    
    b_prim -= c[i,i]*x[i]    
        
    ell[i] = n_bits[i] / np.log2(1 + a[i] * x[i] / (b[i] * x[i] + b_prim + d[i]))
    
    E[i] = x_u*x[i]*ell[i]
    
    
print('latency:', ell) 
print('max latency:', np.max(ell))
print('----------------')
print('energy:', E)

print('sum energy:', np.sum(E))

Optimal solution: [0.42448208 0.33260697 0.76016006]
latency: [[7.88819628]
 [4.75468935]
 [4.60266859]]
max latency: 7.888196279759467
----------------
energy: [[0.3348398 ]
 [0.15814428]
 [0.34987648]]
sum energy: 0.8428605596584361


C:\Users\afmb\AppData\Local\Temp\ipykernel_9324\2394127775.py:6: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  sum_term = 1*sum(n_bits[k]*x[k]*x_u / np.log2(1 + a[k] * x[k] / (b[k] * x[k] + np.sum(c[k, j] * x[j] for j in range(len(x))) - c[k, k] * x[k] + d[k])) for k in range(len(x)))
C:\Users\afmb\AppData\Local\Temp\ipykernel_9324\2394127775.py:8: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  max_term = 0.5*np.sum(n_bits[i] /np.log2(1+ a[i] * x[i] / (b[i] * x[i] + np.sum(c[i, j] * x[j] for j in range(len(x))) - c[i, i] * x[i] + d[i])) for i in range(len(x)))
